In [1]:
import pandas as pd 
import numpy as np 
import os
from unidecode import unidecode
from datetime import datetime, date, timedelta
import xlrd
from xlrd.xldate import xldate_as_tuple
import ConeccionesSucursales as cs
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
ruta = '/home/estadistico/Documents/Erick/'
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
mes = 20210101
mes2 = 'Enero2021'
mes3 = pd.to_datetime('01/01/2021',dayfirst=True)
fin_mes = pd.to_datetime('31/01/2021',dayfirst=True)
quetzal = 2.54
dolar = 19.8

# Lectura Pagos

In [3]:
#Lectura pagos Bradesco
book = os.listdir(''+ruta+'/'+str(mes2)+'/Bradesco/Pagos/')
pagos_brad = pd.DataFrame()
for i in book:
    pagos_new = pd.read_excel(''+ruta+'/'+str(mes2)+'/Bradesco/Pagos/'+i+'')
    pagos_new = pagos_new.iloc[:,0:9]
    pagos_brad = pd.concat([pagos_brad,pagos_new],ignore_index=True)
    pagos_brad = pagos_brad.drop_duplicates()
pagos_brad = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
pagos_brad = pd.DataFrame(pagos_brad.to_records())
pagos_brad = pagos_brad.loc[pagos_brad['FECHA_PAGO']>=mes3]

In [4]:
#Lectura pagos Liverpool
book =  os.listdir(''+ruta+'/'+str(mes2)+'/Liverpool/Pagos/')
sheet_liv = ['SA002REM','SA001REM','DKN04REM','DKN02REM','DKN00REM']
pagos_liv = pd.DataFrame()
pagos_liv_new = pd.DataFrame()
for i in book:
    for j in sheet_liv:
        pagos_new = pd.read_excel(''+ruta+'/'+str(mes2)+'/Liverpool/Pagos/'+i+'',sheet_name = str(j),converters={'Cuenta':str})
        pagos_new.columns = ['Id','Cuenta','Fecha Prom','Fecha Pago','Monto','Posteo']
        pagos_liv_new = pd.concat([pagos_liv_new,pagos_new],ignore_index=True)
        pagos_liv_new = pagos_liv_new.iloc[:,0:6]
    pagos_liv = pd.concat([pagos_liv,pagos_liv_new],ignore_index=True)
    pagos_liv = pagos_liv.drop_duplicates()
pagos_liv['Fecha Pago'] = pd.to_datetime(pagos_liv['Fecha Pago'],dayfirst=True)
pagos_liv = pd.pivot_table(pagos_liv,index=['Id','Cuenta','Fecha Pago','Fecha Prom'],values=['Monto'],aggfunc=np.sum)
pagos_liv = pd.DataFrame(pagos_liv.to_records())
pagos_liv = pagos_liv.loc[pagos_liv['Fecha Pago']>=mes3]

In [5]:
#Lectura pagos Credifiel
book = os.listdir(''+ruta+'/'+str(mes2)+'/Credifiel/Pagos/')
pagos_cred = pd.DataFrame()
for j in book:
    pagos_new = pd.read_excel(''+ruta+'/'+str(mes2)+'/Credifiel/Pagos/'+j+'')
    pagos_cred = pd.concat([pagos_cred,pagos_new],ignore_index=True)
#pagos_cred = pagos_cred.iloc[:,[12,1,8]]
pagos_cred = pd.pivot_table(pagos_cred,index=['ID','FECHA'],values=['IMPORTE'],aggfunc=np.sum)
pagos_cred = pd.DataFrame(pagos_cred.to_records())
pagos_cred = pagos_cred.loc[pagos_cred['FECHA']>=mes3]

In [6]:
#Lectura pagos Banco Azteca Renta
book = os.listdir(''+ruta+'/'+str(mes2)+'/Banco Azteca/Pagos/Renta/')
pagos_baz_renta = pd.DataFrame()
for i in book:
    pagos_new = pd.read_csv(''+ruta+'/'+str(mes2)+'/Banco Azteca/Pagos/Renta/'+i+'')
    pagos_baz_renta = pd.concat([pagos_baz_renta,pagos_new],ignore_index=True)
pagos_baz_renta = pagos_baz_renta.drop_duplicates()
pagos_baz_renta = pagos_baz_renta.loc[pagos_baz_renta['FNMONTOPAGADO']>0]
pagos_baz_renta['FDFECHAABONO'] = pd.to_datetime(pagos_baz_renta['FDFECHAABONO'],dayfirst=True)
pagos_baz_renta = pagos_baz_renta.loc[pagos_baz_renta['FDFECHAABONO']>=mes3]
pagos_baz_renta = pd.pivot_table(pagos_baz_renta,index=['CAMPANAID','FDFECHAABONO','FIIDPERIODO','FNSCOMPROMISO','FCEMPNUMCORTE','FNMONTOPROMETIDO'],values=['FNMONTOPAGADO'],aggfunc=np.sum)
pagos_baz_renta = pd.DataFrame(pagos_baz_renta.to_records())
pagos_baz_renta = pagos_baz_renta.loc[pagos_baz_renta['FDFECHAABONO']>=mes3]
pagos_baz_renta['Cliente'] = 'Banco Azteca'
pagos_baz_renta['Division'] = 'Renta'

In [7]:
#Lectura pagos Banco Azteca Recovery
book = os.listdir(''+ruta+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/')
pagos_baz_recovery = pd.DataFrame()
for i in book:
    pagos_new = pd.read_csv(''+ruta+'/'+str(mes2)+'/Banco Azteca/Pagos/Recovery/'+i+'',sep='|')
    pagos_baz_recovery = pd.concat([pagos_baz_recovery,pagos_new],ignore_index=True)
pagos_baz_recovery['DIA'] = pd.to_datetime(pagos_baz_recovery['DIA'],dayfirst=True)
pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
pagos_baz_recovery = pagos_baz_recovery.loc[pagos_baz_recovery['DIA']>=mes3]
pagos_baz_recovery['credito'] = pagos_baz_recovery['PAIS'].map(str)+'-'+pagos_baz_recovery['CANAL'].map(str)+'-'+pagos_baz_recovery['SUCURSAL'].map(str)+'-'+pagos_baz_recovery['FOLIO'].map(str)
pagos_baz_recovery = pagos_baz_recovery.drop_duplicates()
pagos_baz_recovery['Cliente'] = 'Banco Azteca'
pagos_baz_recovery['Division'] = 'Renta'

In [8]:
#Lectura pagos Banco Azteca 26-39
book = os.listdir(''+ruta+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/')
pagos_baz_26 = pd.DataFrame()
for i in book:
    pagos_new = pd.read_csv(''+ruta+'/'+str(mes2)+'/Banco Azteca/Pagos/26-39/'+i+'',sep='|')
    pagos_baz_26 = pd.concat([pagos_baz_26,pagos_new],ignore_index=True)
pagos_baz_26['DIA'] = pd.to_datetime(pagos_baz_26['DIA'],dayfirst=True)
pagos_baz_26 = pagos_baz_26.loc[pagos_baz_26['DIA']>=mes3]
pagos_baz_26['credito'] = pagos_baz_26['PAIS'].map(str)+'-'+pagos_baz_26['CANAL'].map(str)+'-'+pagos_baz_26['SUCURSAL'].map(str)+'-'+pagos_baz_26['FOLIO'].map(str)
pagos_baz_26 = pagos_baz_26.drop_duplicates()
pagos_baz_26['Cliente'] = 'Banco Azteca'
pagos_baz_26['cartera'] = '26-39'

In [9]:
#Leemos facturacion Banco Azteca
book = os.listdir(''+ruta+'/'+str(mes2)+'/Banco Azteca/Facturacion/')
fact_baz = pd.DataFrame()
for i in book:
    pagos_new = pd.read_csv(''+ruta+'/'+str(mes2)+'/Banco Azteca/Facturacion/'+i+'',encoding='utf-8')
    fact_baz = pd.concat([fact_baz,pagos_new],ignore_index=True)
fact_baz = fact_baz.drop_duplicates()
fact_baz['Final de Semana'] = pd.to_datetime(fact_baz['Final de Semana'],dayfirst=True)
fact_baz['Inicio de Semana'] = pd.to_datetime(fact_baz['Inicio de Semana'],dayfirst=True)
fact_baz['Cliente'] = 'Banco Azteca'

In [10]:
#Lectura pagos Oriflam
book = os.listdir(''+ruta+'/'+mes2+'/Torreon/Oriflame/Pagos/')
pagos_oriflame = pd.DataFrame()
for i in book:
    pagos_new = pd.read_excel(''+ruta+'/'+mes2+'/Torreon/Oriflame/Pagos/'+i+'')
    pagos_oriflame = pd.concat([pagos_oriflame,pagos_new],ignore_index=True)
    pagos_oriflame = pagos_oriflame.drop_duplicates()
pagos_oriflame['Cliente'] = 'Oriflame'
pagos_oriflame['Factura'] = pagos_oriflame['PAGO_TOTAL']*.1
pagos_oriflame = pagos_oriflame.loc[pagos_oriflame['FECHA_PAGO']>=mes3]

In [11]:
#Lectura pagos Arabela
book = os.listdir(''+ruta+'/'+mes2+'/Torreon/Arabela/Pagos/')
pagos_arabela = pd.DataFrame()
for i in book:
    pagos_new = pd.read_excel(''+ruta+'/'+mes2+'/Torreon/Arabela/Pagos/'+i+'')
    pagos_arabela = pd.concat([pagos_arabela,pagos_new],ignore_index=True)
    pagos_arabela = pagos_arabela.drop_duplicates()
pagos_arabela['Cliente'] = 'Arabela'
pagos_arabela['FechaEntrega'] = [datetime.strptime(str(x), '%Y%m%d').strftime('%d/%m/%Y') for x in pagos_arabela['FechaEntrega']]
pagos_arabela['FechaEntrega'] = pd.to_datetime(pagos_arabela['FechaEntrega'],dayfirst=True)
pagos_arabela = pagos_arabela.loc[pagos_arabela['FechaEntrega']>=mes3]
pagos_arabela['factura'] = pagos_arabela['pago']*.22

In [12]:
#Lectura pagos invex
book = os.listdir(''+ruta+''+mes2+'/Torreon/Invex/Pagos/')
pagos_invex = pd.DataFrame()
for i in book:
    pagos_new = pd.read_excel(''+ruta+'/'+mes2+'/Torreon/Invex/Pagos/'+i+'')
    pagos_invex = pd.concat([pagos_invex,pagos_new],ignore_index=True)
pagos_invex['Cliente'] = 'Invex_'+pagos_invex['Agencia'].map(str)
pagos_invex = pagos_invex.drop_duplicates()
# pagos_invex = pagos_invex.loc[pagos_invex['TRAN_POST_DT']>=mes3]

# Lectura asignaciones

In [13]:
#Asignacion Liverpool
asignacion_liv = pd.read_csv(''+ruta+'/Liverpool/Asignacion/Asignaciones csv/Asignacion '+mes2+'.csv',converters={'# CUENTA':str})
asignacion_liv = asignacion_liv[['ID AGENCIA', '# CUENTA']]
asignacion_liv.columns = ['division','credito']

In [14]:
#Asignacion Bradesco
asignacion_brad_c0 = pd.read_csv(''+ruta+'/Bradesco/Asignacion_c0/CSV/ASIGNACION C0 '+mes2+'.csv')
asignacion_brad_c0 = asignacion_brad_c0.iloc[:,[0,2,3,8,10,11,14,25]]
asignacion_brad_extra = pd.read_csv(''+ruta+'/Bradesco/Asignacion_extra/CSV/Asignacion Extra '+mes2+'.csv',encoding= 'unicode_escape')
asignacion_brad_extra = asignacion_brad_extra.iloc[:,[0,2,3,5,9,25,26,28]]
asignacion_brad_extra.columns = ['CODIGO_ORG','NUMERO_CUENTA','NUMERO_TARJETA','RFC','ESTADO','SALDO_TOTAL','PAGO_DESCUENTO','PAGOSVENCIDOS']

asignacion_brad = pd.concat([asignacion_brad_c0,asignacion_brad_extra],ignore_index=True)

In [15]:
#Hacemos la consulta de la asignacion
asignacion_cred = pd.read_excel(''+ruta+'/Credifiel/Asignacion/Asignacion Credifiel '+mes2+'.xlsx')
asignacion_cred = asignacion_cred.drop_duplicates(subset=['CREDITO','SALDO TOTAL REAL'])
asignacion_cred['CREDITO'] = pd.to_numeric(asignacion_cred['CREDITO'])

In [16]:
# #Asignacion Banco Azteca Renta
# book = os.listdir(''+ruta+'/'+mes2+'/Banco Azteca/Asignacion/Renta/')
# asignacion_renta = pd.DataFrame()
# for i in book:
#     asignacion_new = pd.read_excel(''+ruta+'/'+mes2+'/Banco Azteca/Asignacion/Renta/'+i+'')
#     asignacion_renta = pd.concat([asignacion_renta,asignacion_new],ignore_index=True)

# Parametros a ajustar

In [17]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
c04pv1, arancel_4pv1 = 1204704, .07
c04pv2, arancel_4pv2 = 1807056, .09
c04pv3, arancel_4pv3 = 2409407, .12
c05pv1, arancel_5pv1 = 662269, .09
c05pv2, arancel_5pv2 = 993403, .10
c05pv3, arancel_5pv3 = 1324538, .14
c06pv1, arancel_6pv1 = 807172, .10
c06pv2, arancel_6pv2 = 1210759, .11
c06pv3, arancel_6pv3 = 1614345, .16
a1, arancel_a1 = 260190, .27
a2, arancel_a2 = 390285, .30
a3, arancel_a3 = 520381, .33
b1, arancel_b1 = 215083, .25
b2, arancel_b2 = 322624, .28
b3, arancel_b3 = 430165, .31
c11, arancel_c11 = 1430989, .17
c12, arancel_c12 = 2146483, .20
c13, arancel_c13 = 2861978, .23
c21, arancel_c21 = 626634, .19
c22, arancel_c22 = 939951, .22
c23, arancel_c23 = 1253267, .25
c31, arancel_c31 = 246425, .21
c32, arancel_c32 = 369637, .24
c33, arancel_c33 = 492849, .27
c41, arancel_c41 = 123529, .23
c42, arancel_c42 = 185294, .26
c43, arancel_c43 = 247059, .29

In [18]:
#Hacemos la asignacion de cada una de las metas para cambiar de arancel sin tomar el 0 en cuenta
cof, arancel_cof,arancel_cof2 = 300000, .5, .5
sub, arancel_sub, arancel_sub2 = 300000, .15, .2
subdic, arancel_subdic, arancel_subdic2 = 350000, .16, .2
n2, arancel_n2, arancel_n22 = 650000, .4, .5
pren, arancel_pren, arancel_pren2 = 300000, .28, .28

In [19]:
#Metas Naucalpan
meta_naucalpan = pd.DataFrame([['Liverpool', 1150000],['Liverpool Cof', 300000],['Liverpool Nacional', 550000],['Bradesco',750000],['Credifiel',100000],['Banco Azteca',1000000]])
meta_naucalpan.columns = ['Cliente','Meta']
#Metas Torreon
meta_torreon = pd.DataFrame([('Avon',990000),('Oriflame',65000),('Invex_RLN',270000),('Invex_RLA',479000),('Invex_REL',470000),('Arabela',95000),('Cristian Lay',6000)])
meta_torreon.columns = ['Cliente','Meta']

# Analisis

In [20]:
#Asignamos valores a los parametros \n"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H-%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

In [21]:
#Funcion para transformar varias columnas a numero
def transform_numeric(df,cols):
    return df[cols].apply(pd.to_numeric, errors='coerce')

#Funcion que convierte a dataframe y rellena espacios con 0
def to_dataframe(df):
    df = pd.DataFrame(df.to_records())
    df.fillna(0,inplace=True)
    return df

# $\textbf{Naucalpan}$

# Liverpool

In [22]:
#Hacemos la consulta referente a las gestiones de Liverpool
gestion_liv_mes = cs.query_naucalpan('liverpool_rpt_gestiones_detallado',mes,today)
gestion_liv_mes.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv_mes = gestion_liv_mes.loc[gestion_liv_mes['dictamen']=='PROMESA']
gestion_liv_mes['anio'] = [int(x[:4]) for x in gestion_liv_mes['fecha_promesa']]
gestion_liv_mes = gestion_liv_mes.loc[(gestion_liv_mes['anio']>2010)&(gestion_liv_mes['anio']<2100)]
gestion_liv_mes['fecha_promesa'] = pd.to_datetime(gestion_liv_mes['fecha_promesa'],dayfirst=False)

In [23]:
#Hacemos el consolidado
consolidado_liv_mes = pd.merge(gestion_liv_mes,asignacion_liv,how='left',on=['credito'])
cols = ['monto_promesa']
consolidado_liv_mes[cols] = consolidado_liv_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']>50)]
consolidado_liv_mes = consolidado_liv_mes[(consolidado_liv_mes['monto_promesa']<500000)]
consolidado_liv_mes['division'].fillna('COBRANZA',inplace=True)
consolidado_liv_mes['division'] = consolidado_liv_mes['division'].replace('','COBRANZA')
consolidado_liv_mes = consolidado_liv_mes.drop_duplicates(subset = ['usuario','credito','fecha_promesa','monto_promesa'])

In [24]:
productos = [('DKN00REM','Liverpool','Penacional'),('DKN02REM','Liverpool Nacional','Nacional2'),('DKN04REM','Liverpool Cof','Cof'),('SA001REM','Liverpool','Suburbia'),('SA002REM','Liverpool','Dictamen')]
productos = pd.DataFrame(productos)
productos.columns = ['division','id','segmento']
consolidado_liv_mes = pd.merge(consolidado_liv_mes,productos,how='left',on=['division'])

In [25]:
#Obtenemos los pagos totales por division
pagos_totales_liv = pd.pivot_table(pagos_liv,index=['Id'],values=['Monto'],aggfunc=[np.sum])
pagos_totales_liv = pd.DataFrame(pagos_totales_liv.to_records())
#pagos_totales_liv = pagos_totales_liv2.iloc[:,[0,2,1]]
pagos_totales_liv.columns = ['Division','SumaPagos']
pagos_totales_liv.fillna(0,inplace=True)

In [26]:
#Para cada segmento se le asigna lo que lleva de cada uno
DKN00REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='DKN00REM', 'SumaPagos'].item()
DKN02REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='DKN02REM', 'SumaPagos'].item()
DKN04REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='DKN04REM', 'SumaPagos'].item()
SA001REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='SA001REM', 'SumaPagos'].item()
SA002REM = pagos_totales_liv.loc[pagos_totales_liv['Division']=='SA002REM', 'SumaPagos'].item()

In [27]:
#Funcion que asignará el arancel correspondiente
def arancel_liv(row):
    if row['Id'] == 'DKN00REM' :
        return np.where((DKN00REM>0) & (DKN00REM<=pren/arancel_pren),arancel_pren,arancel_pren2)
    elif row['Id'] == 'DKN02REM':
        return np.where((DKN02REM>0) & (DKN02REM<=n2/arancel_n2),arancel_n2,arancel_n22)
    elif row['Id'] == 'DKN04REM':
        return np.where((DKN04REM>0) & (DKN04REM<=cof/arancel_cof),arancel_cof,arancel_cof2)
    elif row['Id'] == 'SA001REM':
        return np.where((SA001REM>0) & (SA001REM<=sub/arancel_sub),arancel_sub,arancel_sub2)
    elif row['Id'] == 'SA002REM':
        return np.where((SA002REM>0) & (SA002REM<=subdic/arancel_subdic),arancel_subdic,arancel_subdic2)
    else:
        return 0

In [28]:
#Aplicamos la funcion para obtener los aranceles
pagos_liv2 = pd.merge(pagos_liv,productos,how='left',left_on='Id',right_on='division')
pagos_liv2['Arancel'] = pagos_liv2.apply(arancel_liv, axis=1)

#Agregamos aranceles al consolidado
pagos_liv2['PagoArancel'] = [x*y for x,y in zip(pagos_liv2['Monto'],pagos_liv2['Arancel'])]
pagos_liv2['Fecha Pago'] = pd.to_datetime(pagos_liv2['Fecha Pago'],dayfirst=True)
pagos_liv2.head(1)

,Id,Cuenta,Fecha Pago,Fecha Prom,Monto,division,id,segmento,Arancel,PagoArancel
0,DKN00REM,0000013000000133855,2021-01-08,2021-01-07 00:00:00,4797.0,DKN00REM,Liverpool,Penacional,0.28,1343.16


In [29]:
#Total pagos Liver registrados
pagos_liv2['Cliente'] = 'Liverpool'
pagos_totales_liv2 = pd.pivot_table(pagos_liv2,index=['Cliente'],values=['PagoArancel'],aggfunc=np.sum,dropna=False)
pagos_totales_liv2 = pd.DataFrame(pagos_totales_liv2.to_records())
#pagos_totales_liv = pagos_totales_liv.iloc[:,[0,3,1,2]]
pagos_totales_liv2.columns = ['Cliente','Facturacion']
pagos_totales_liv2.fillna(0,inplace=True)
pagos_totales_liv2['Facturacion'] = pagos_totales_liv2['Facturacion'].round(2)
pagos_totales_liv2['FechaFact'] = max(pagos_liv2['Fecha Pago'])
pagos_totales_liv2['FechaFact'] = pd.to_datetime(pagos_totales_liv2['FechaFact'],dayfirst=True)
pagos_totales_liv2

,Cliente,Facturacion,FechaFact
0,Liverpool,370631.72,2021-01-17


In [30]:
#Funcion que asigna el siguiente arancel
def sig_arancel_liv(row):
    if row['Asignacion'] == 'DKN00REM' :
        return np.where((DKN00REM>0) & (DKN00REM<=pren/arancel_pren),round(pren/arancel_pren,1),0)
    elif row['Asignacion'] == 'DKN02REM':
        return np.where((DKN02REM>0) & (DKN02REM<=n2/arancel_n2),n2/arancel_n2,0)
    elif row['Asignacion'] == 'DKN04REM':
        return np.where((DKN04REM>0) & (DKN04REM<=cof/arancel_cof),cof/arancel_cof,0)
    elif row['Asignacion'] == 'SA001REM':
        return np.where((SA001REM>0) & (SA001REM<=sub/arancel_sub),sub/arancel_sub,0)
    elif row['Asignacion'] == 'SA002REM':
        return np.where((SA002REM>0) & (SA002REM<=subdic/arancel_subdic),subdic/arancel_subdic,0)
    else:
        return 0

In [31]:
#Aplicamos 
pagos_liv_gerencia = pd.pivot_table(pagos_liv2,index=['Id'],values=['Monto'],aggfunc=[np.sum])
pagos_liv_gerencia = pd.DataFrame(pagos_liv_gerencia.to_records())
pagos_liv_gerencia.columns = ['Asignacion','Recuperado']
pagos_liv_gerencia['SiguienteArancel'] = pagos_liv_gerencia.apply(sig_arancel_liv, axis=1)
pagos_liv_gerencia['RestSigArancel'] = pagos_liv_gerencia['SiguienteArancel']-pagos_liv_gerencia['Recuperado']
cols = ['Recuperado','SiguienteArancel','RestSigArancel']
pagos_liv_gerencia[cols] = pagos_liv_gerencia[cols].round(2)
pagos_liv_gerencia['RestSigArancel'] = np.where(pagos_liv_gerencia['RestSigArancel']<0,'Alcanzado',pagos_liv_gerencia['RestSigArancel'])
pagos_liv_gerencia = pd.merge(pagos_liv_gerencia,productos,how='left',left_on='Asignacion',right_on='division')
pagos_liv_gerencia = pagos_liv_gerencia.iloc[:,[6,1,2,3]]
pagos_liv_gerencia.columns = ['Division','Recuperado','SiguienteArancel','RestSigArancel']

In [32]:
#Unimos el consolidado con los pagos 
consolidado_pagos_liv = pd.merge(consolidado_liv_mes,pagos_liv2[['Id','Cuenta','Fecha Pago','Monto','id','segmento','Arancel','PagoArancel']],how='left',left_on=['credito'],right_on=['Cuenta'])
consolidado_pagos_liv = consolidado_pagos_liv.drop_duplicates(subset = ['credito','Monto','usuario','fecha_promesa'])
consolidado_pagos_liv['Fecha Pago'] = pd.to_datetime(consolidado_pagos_liv['Fecha Pago'],dayfirst=True)
consolidado_pagos_liv['dia_prom'] = consolidado_pagos_liv['fecha_promesa'].dt.day
consolidado_pagos_liv['dia_pago'] = consolidado_pagos_liv['Fecha Pago'].dt.day
cols = ['dia_prom','dia_pago','Monto']
consolidado_pagos_liv[cols] = consolidado_pagos_liv[cols].apply(pd.to_numeric, errors='coerce',downcast='integer')
consolidado_pagos_liv['dia_pago'].fillna(0,inplace=True)
consolidado_pagos_liv['dia_pago'] = [int(x) for x in consolidado_pagos_liv['dia_pago']]

In [33]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
consolidado_pagos_liv['PagoReal'] = [np.where((z < x+pd.Timedelta(3, unit='D')) & ((z > x-pd.Timedelta(3, unit='D') )), y, 0) for x,y,z in zip(consolidado_pagos_liv['fecha_promesa'],consolidado_pagos_liv['Monto'],consolidado_pagos_liv['Fecha Pago'])]
consolidado_pagos_liv['PagoReal'] = [x.item() for x in consolidado_pagos_liv['PagoReal']]
consolidado_pagos_liv['MontoArancel'] = [x*y for x,y in zip(consolidado_pagos_liv['PagoReal'],consolidado_pagos_liv['Arancel'])]

In [34]:
#Total pagos Liver registrados
pagos_totales_liv3 = pd.pivot_table(pagos_liv2,index=['segmento'],values=['PagoArancel'],aggfunc=np.sum,dropna=False)
pagos_totales_liv3 = pd.DataFrame(pagos_totales_liv3.to_records())
#pagos_totales_liv = pagos_totales_liv.iloc[:,[0,3,1,2]]
pagos_totales_liv3.columns = ['Division','Facturacion']
pagos_totales_liv3.fillna(0,inplace=True)
pagos_totales_liv3['Facturacion'] = pagos_totales_liv3['Facturacion'].round(2)
pagos_totales_liv3['FechaFact'] = max(pagos_liv2['Fecha Pago'])
pagos_totales_liv3['FechaFact'] = pd.to_datetime(pagos_totales_liv3['FechaFact'],dayfirst=True)
pagos_totales_liv3

,Division,Facturacion,FechaFact
0,Cof,90077.68,2021-01-17
1,Dictamen,2672.16,2021-01-17
2,Nacional2,142921.20,2021-01-17
3,Penacional,111875.68,2021-01-17
4,Suburbia,23085.00,2021-01-17


In [35]:
#Obtenemos datos de los gestores de Liverpool
promesas_mes_liv = pd.pivot_table(consolidado_pagos_liv,index=['segmento_y'],values=['monto_promesa'],aggfunc=['count',np.sum],dropna=False)
promesas_mes_liv = pd.DataFrame(promesas_mes_liv.to_records())
promesas_mes_liv.columns = ['Division','NumeroPromesas','SumaPromesas']
promesas_mes_liv.fillna(0,inplace=True)
promesas_mes_liv

,Division,NumeroPromesas,SumaPromesas
0,Cof,52,185138.37
1,Dictamen,7,17875.00
2,Nacional2,79,489763.72
3,Penacional,112,433530.00
4,Suburbia,112,188623.74


In [36]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_liv = consolidado_pagos_liv.loc[(consolidado_pagos_liv['fecha_promesa']>=ayer)&(consolidado_pagos_liv['fecha_promesa']<=fin_mes)]
promesas_restantes_liv = pd.pivot_table(restantes_liv,index=['segmento_y'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_liv = pd.DataFrame(promesas_restantes_liv.to_records())
promesas_restantes_liv.columns = ['Division','PromesasRestantes']
promesas_mes_liv.fillna(0,inplace=True)
promesas_restantes_liv

,Division,PromesasRestantes
0,Cof,25326.0
1,Nacional2,76747.0
2,Penacional,75873.0
3,Suburbia,56337.1


In [37]:
pagos_totales_liv2

,Cliente,Facturacion,FechaFact
0,Liverpool,370631.72,2021-01-17


In [84]:
final_liv = pd.merge(pagos_totales_liv3,pagos_liv_gerencia,how='left',on='Division')
final_liv = pd.merge(final_liv,promesas_restantes_liv,how='left',on='Division')
#final_liv.insert(loc=0,column='Cliente',value='Liverpool')
final_liv.drop('SiguienteArancel',axis=1,inplace=True)
final_liv.insert(loc=0,column='Cliente',value='Liverpool')
final_liv

,Cliente,Division,Facturacion,FechaFact,Recuperado,RestSigArancel,PromesasRestantes
0,Liverpool,Cof,90077.68,2021-01-17,180155.37,419845,25326.0
1,Liverpool,Dictamen,2672.16,2021-01-17,16701.00,2.1708e+06,NaN
2,Liverpool,Nacional2,142921.20,2021-01-17,357302.99,1.2677e+06,76747.0
3,Liverpool,Penacional,111875.68,2021-01-17,399556.00,671873,75873.0
4,Liverpool,Suburbia,23085.00,2021-01-17,153899.98,1.8461e+06,56337.1


# Bradesco

In [39]:
#Hacemos la consulta referente a las gestiones de Bradesco
gestion_brad_mes = cs.query_naucalpan('bradescard_rpt_gestiones_detallado',mes,today)
gestion_brad_mes.columns = ['folio_gestion','firma_id','unegocio_id','NUMERO_TARJETA','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad_mes = gestion_brad_mes.loc[gestion_brad_mes['dictamen']=='PROMESA']
gestion_brad_mes['NUMERO_TARJETA'] = pd.to_numeric(gestion_brad_mes['NUMERO_TARJETA'])
gestion_brad_mes['monto_promesa'] = pd.to_numeric(gestion_brad_mes['monto_promesa'])
gestion_brad_mes = gestion_brad_mes.drop_duplicates(subset=['NUMERO_TARJETA','usuario','fecha_promesa','monto_promesa'])
gestion_brad_mes = gestion_brad_mes.loc[(gestion_brad_mes['fecha_promesa'].notnull())&(gestion_brad_mes['fecha_promesa']!='')]
gestion_brad_mes = gestion_brad_mes.loc[(gestion_brad_mes['fecha_promesa'].str[:4].map(int)<=2100)&(gestion_brad_mes['fecha_promesa'].str[:4].map(int)>=2000)]
gestion_brad_mes['fecha_promesa'] = pd.to_datetime(gestion_brad_mes['fecha_promesa'],dayfirst=False)

In [40]:
#Hacemos la union de asignacion y promesas
consolidado_brad_mes = pd.merge(gestion_brad_mes,asignacion_brad,how='left',on=['NUMERO_TARJETA'])
cols = ['monto_promesa']
consolidado_brad_mes[cols] = consolidado_brad_mes[cols].apply(pd.to_numeric, errors='coerce')
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']>50)]
consolidado_brad_mes = consolidado_brad_mes[(consolidado_brad_mes['monto_promesa']<500000)]
#consolidado_brad_mes.fillna('COBRANZA',inplace=True)
consolidado_brad_mes['NUMERO_TARJETA'] = pd.to_numeric(consolidado_brad_mes['NUMERO_TARJETA'])

In [41]:
#Obtenemos los pagos totales por division
pagos_totales_brad = pd.pivot_table(pagos_brad,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'],margins=True,dropna=False)
pagos_totales_brad = pd.DataFrame(pagos_totales_brad.to_records())
pagos_totales_brad = pagos_totales_brad.iloc[:,[0,2,1]]
pagos_totales_brad.columns = ['Division','NumeroPagos','SumaPagos']
pagos_totales_brad.fillna(0,inplace=True)

In [42]:
#Obtenemos el valor para cada segmento
C04PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 4PV', 'SumaPagos'].item()
C05PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 5PV', 'SumaPagos'].item()
C06PV = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C0 6PV', 'SumaPagos'].item()
A = pagos_totales_brad.loc[pagos_totales_brad['Division']=='A', 'SumaPagos'].item()
C1 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C1', 'SumaPagos'].item()
C2 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C2', 'SumaPagos'].item()
C3 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C3', 'SumaPagos'].item()
C4 = pagos_totales_brad.loc[pagos_totales_brad['Division']=='C4', 'SumaPagos'].item()
B = pagos_totales_brad.loc[pagos_totales_brad['Division']=='B', 'SumaPagos'].item()

In [43]:
#Funcion para asignar siguiente arancel
def sig_arancel(row):
    if row['Asignacion'] == 'C0 4PV':
        return np.where((C04PV>0) & (C04PV<=c04pv1),c04pv1,
                        np.where((C04PV>c04pv1) & (C04PV<=c04pv2),c04pv2,c04pv3))
    elif row['Asignacion'] == 'C0 5PV':
        return np.where((C05PV>0) & (C05PV<=c05pv1),c05pv1,
                        np.where((C05PV>c05pv1) & (C05PV<=c05pv2),c05pv2,c05pv3))
    elif row['Asignacion'] == 'C0 6PV':
        return np.where((C06PV>0) & (C06PV<=c06pv1),c06pv1,
                        np.where((C06PV>c06pv1) & (C06PV<=c06pv2),c06pv2,c06pv3))
    elif row['Asignacion'] == 'A':
        return np.where((A>0) & (A<=a1),a1,
                        np.where((A>a1) & (A<=a2),a2,a3))
    elif row['Asignacion'] == 'B':
        return np.where((B>0) & (B<=b1),b1,
                        np.where((B>b1) & (B<=b2),b2,b3))
    elif row['Asignacion'] == 'C1':
        return np.where((C1>0) & (C1<=c11),c11,
                        np.where((C1>c11) & (C1<=c12),c12,c13))
    elif row['Asignacion'] == 'C2':
        return np.where((C2>0) & (C2<=c21),c21,
                        np.where((C2>c21) & (C2<=c22),c22,c23))
    elif row['Asignacion'] == 'C3':
        return np.where((C3>0) & (C3<=c31),c31,
                        np.where((C3>c31) & (C3<=c32),c32,c33))
    elif row['Asignacion'] == 'C4':
        return np.where((C4>0) & (C4<=c41),c41,
                        np.where((C4>c41) & (C4<=c42),c42,c43))
    
    else:
        return 0

In [44]:
#Funcion para asignar arancel
def func(row):
    if row['TIPO_ASIGANCION'] == 'C0 4PV':
        return np.where((C04PV>0) & (C04PV<=c04pv1),arancel_4pv1,
                        np.where((C04PV>c04pv1) & (C04PV<=c04pv2),arancel_4pv2,
                                 np.where((C04PV>c04pv2) & (C04PV<=c04pv3),arancel_4pv3,arancel_4pv3+.04)))
    elif row['TIPO_ASIGANCION'] == 'C0 5PV':
        return np.where((C05PV>0) & (C05PV<=c05pv1),arancel_5pv1,
                        np.where((C05PV>c05pv1) & (C05PV<=c05pv2),arancel_5pv2,
                                 np.where((C05PV>c05pv2) & (C05PV<=c05pv3),arancel_5pv3,arancel_5pv3+.04)))
    elif row['TIPO_ASIGANCION'] == 'C0 6PV':
        return np.where((C06PV>0) & (C06PV<=c06pv1),arancel_6pv1,
                        np.where((C06PV>c06pv1) & (C06PV<=c06pv2),arancel_6pv2,
                                 np.where((C06PV>c06pv2) & (C06PV<=c06pv3),arancel_6pv3,arancel_6pv3+.04)))
    elif row['TIPO_ASIGANCION'] == 'A':
        return np.where((A>0) & (A<=a1),arancel_a1,
                        np.where((A>a1) & (A<=a2),arancel_a2,
                                 np.where((A>a2) & (A<=a3),arancel_a3,arancel_a3+.04)))
    elif row['TIPO_ASIGANCION'] == 'B':
        return np.where((B>0) & (B<=b1),arancel_b1,
                        np.where((B>b1) & (B<=b2),arancel_b2,
                                 np.where((B>b2) & (B<=b3),arancel_b3,arancel_b3+.04)))
    elif row['TIPO_ASIGANCION'] == 'C1':
        return np.where((C1>0) & (C1<=c11),arancel_c11,
                        np.where((C1>c11) & (C1<=c12),arancel_c12,
                                 np.where((C1>c12) & (C1<=c13),arancel_c13,arancel_c13+.04)))
    elif row['TIPO_ASIGANCION'] == 'C2':
        return np.where((C2>0) & (C2<=c21),arancel_c21,
                        np.where((C2>c21) & (C2<=c22),arancel_c22,
                                 np.where((C2>c22) & (C2<=c23),arancel_c23,arancel_c23+.04)))
    elif row['TIPO_ASIGANCION'] == 'C3':
        return np.where((C3>0) & (C3<=c31),arancel_c31,
                        np.where((C3>c31) & (C3<=c32),arancel_c32,
                                 np.where((C3>c32) & (C3<=c33),arancel_c33,arancel_c33+.04)))
    elif row['TIPO_ASIGANCION'] == 'C4':
        return np.where((C4>0) & (C4<=c41),arancel_c41,
                        np.where((C4>c41) & (C4<=c42),arancel_c42,
                                 np.where((C4>c42) & (C4<=c43),arancel_c43,arancel_c43+.04)))
    
    else:
        return 0

In [45]:
#Aplicamos funcion para asignar arancel
pagos_brad2 = pagos_brad
pagos_brad2['Arancel'] = pagos_brad2.apply(func, axis=1)
pagos_brad2['Facturacion'] = [x*y for x,y in zip(pagos_brad2['MONTO'],pagos_brad2['Arancel'])]
pagos_brad2['Cliente'] = 'Bradesco'

In [46]:
pagos_totales_brad2 = pd.pivot_table(pagos_brad2,index=['Cliente'],values=['Facturacion'],aggfunc=[np.sum],dropna=False)
pagos_totales_brad2 = pd.DataFrame(pagos_totales_brad2.to_records())
pagos_totales_brad2.columns = ['Cliente','Facturacion']
pagos_totales_brad2.fillna(0,inplace=True)
pagos_totales_brad2['FechaFact'] = max(pagos_brad2['FECHA_PAGO'])
pagos_totales_brad2

,Cliente,Facturacion,FechaFact
0,Bradesco,176266.4883,2021-01-19


In [96]:
pagos_brad_gerencia = pd.pivot_table(pagos_brad2,index=['TIPO_ASIGANCION'],values=['MONTO','Facturacion'],aggfunc=[np.sum])
pagos_brad_gerencia = pd.DataFrame(pagos_brad_gerencia.to_records())
pagos_brad_gerencia.columns = ['Asignacion','Facturacion','Recuperado']
pagos_brad_gerencia.fillna(0,inplace=True)
pagos_brad_gerencia['SiguienteArancel'] = pagos_brad_gerencia.apply(sig_arancel, axis=1)
pagos_brad_gerencia['RestSigArancel'] = pagos_brad_gerencia['SiguienteArancel']-pagos_brad_gerencia['Recuperado']
cols = ['Facturacion','Recuperado','SiguienteArancel','RestSigArancel']
pagos_brad_gerencia[cols] = pagos_brad_gerencia[cols].round(2)
pagos_brad_gerencia = pagos_brad_gerencia.iloc[:,[0,1,2,4]]
pagos_brad_gerencia.columns = ['Division','Facturacion','Recuperado','RestSigArancel']
pagos_brad_gerencia

,Division,Facturacion,Recuperado,RestSigArancel
0,A,20272.95,75085.00,185105
1,B,6341.00,25364.00,189719
2,C0 4PV,29163.56,416622.32,788082
3,C0 5PV,15198.13,168868.07,493401
4,C0 6PV,14582.45,145824.50,661348
5,C1,37261.62,219186.00,1.2118e+06
6,C2,31644.47,166549.84,460084
7,C3,17422.65,82965.00,163460
8,C4,4379.66,19042.00,104487


In [48]:
#Obtenemos datos de las promesas de Bradesco
# prom_brad_div_mes = pd.pivot_table(consoliado_pagos_brad2,index='PAGOSVENCIDOS',values=['monto_promesa'], aggfunc = ['count',np.sum], margins=True,dropna=False)
# prom_brad_div_mes = pd.DataFrame(prom_brad_div_mes.to_records())
# prom_brad_div_mes.columns = ['Division','NumeroPromesas','SumaPromesas']
# prom_brad_div_mes.fillna(0,inplace=True)
# prom_brad_div_mes

,folio_gestion,firma_id,unegocio_id,NUMERO_TARJETA,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,accion_resultado,fecha_promesa,monto_promesa,comentarios
148,9685869,1,3,4463511465650016,LIMA FLORES ARIATNA MONTSERRAT,5555738688,celular,2021-01-21 07:41:52,TRFIERROSFJ,TRON FIERROS FELIPE DE JESUS,PROMESA,ENTRADA WHATS APP,PROMESA DE PAGO PARCIAL,ENTRADA WHATS APP-PROMESA DE PAGO PARCIAL,2021-01-23,5239.0,5555738688 TT SE REPROGRAMAN PAGOS LIQUIDA CO...


In [49]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_brad = consolidado_brad_mes.loc[(consolidado_brad_mes['fecha_promesa']>=ayer)&(consolidado_brad_mes['fecha_promesa']<=fin_mes)]
promesas_restantes_brad = pd.pivot_table(restantes_brad,index=['PAGOSVENCIDOS'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_brad = pd.DataFrame(promesas_restantes_brad.to_records())
promesas_restantes_brad.columns = ['Division','PromesasRestantes']
promesas_restantes_brad.fillna(0,inplace=True)
promesas_restantes_brad

,Division,PromesasRestantes
0,A,10000.0
1,B,2000.0
2,C0 4PV,207865.0
3,C0 5PV,102281.0
4,C0 6PV,65738.0
5,C1,87276.0
6,C2,73217.0
7,C3,28313.0
8,C4,12477.0


In [97]:
final_brad = pd.merge(pagos_brad_gerencia,promesas_restantes_brad,on='Division',how='left')
final_brad.insert(loc=0,column='Cliente',value='Bradesco')
final_brad['FechaFact'] = max(pagos_brad['FECHA_PAGO'])
final_brad

,Cliente,Division,Facturacion,Recuperado,RestSigArancel,PromesasRestantes,FechaFact
0,Bradesco,A,20272.95,75085.00,185105,10000.0,2021-01-19
1,Bradesco,B,6341.00,25364.00,189719,2000.0,2021-01-19
2,Bradesco,C0 4PV,29163.56,416622.32,788082,207865.0,2021-01-19
3,Bradesco,C0 5PV,15198.13,168868.07,493401,102281.0,2021-01-19
4,Bradesco,C0 6PV,14582.45,145824.50,661348,65738.0,2021-01-19
5,Bradesco,C1,37261.62,219186.00,1.2118e+06,87276.0,2021-01-19
6,Bradesco,C2,31644.47,166549.84,460084,73217.0,2021-01-19
7,Bradesco,C3,17422.65,82965.00,163460,28313.0,2021-01-19
8,Bradesco,C4,4379.66,19042.00,104487,12477.0,2021-01-19


# Credifiel

In [51]:
#Hacemos la consulta referente a las gestiones de Credifiel
gestion_cred = cs.query_naucalpan('credifiel_rpt_gestiones_detallado',mes,today)
gestion_cred.columns = ['folio_gestion','unegocio_id','fecha_gestion','hora_gestion','credito','nombre_credito','telefono','usuario','nombre_usuario','accion','resultado','fecha_promesa','monto_promesa','comentarios','accion_credifiel','resultado_credifiel','dictamen']
gestion_cred['monto_promesa'] = pd.to_numeric(gestion_cred['monto_promesa'])
gestion_cred = gestion_cred.loc[gestion_cred['accion_credifiel']=='PDP']
gestion_cred['credito'] = [x[3:] for x in gestion_cred['credito']]
gestion_cred['credito'] = pd.to_numeric(gestion_cred['credito'])
gestion_cred = gestion_cred.loc[(gestion_cred['fecha_promesa'].notnull())&(gestion_cred['fecha_promesa']!='')]
gestion_cred = gestion_cred.loc[(gestion_cred['fecha_promesa'].str[:4].map(int)<=2100)&(gestion_cred['fecha_promesa'].str[:4].map(int)>=2000)]
gestion_cred['fecha_promesa'] = pd.to_datetime(gestion_cred['fecha_promesa'],dayfirst=False)

In [52]:
##Hacemos el consolidado de la asignacion y promesado
consolidado_cred = pd.merge(gestion_cred,asignacion_cred,how='left',left_on=['credito'],right_on=['CREDITO'])
cols = ['monto_promesa','unegocio_id']
consolidado_cred[cols] = consolidado_cred[cols].apply(pd.to_numeric, errors='coerce')
consolidado_cred = consolidado_cred.loc[consolidado_cred['monto_promesa']>50]
consolidado_cred = consolidado_cred[(consolidado_cred['monto_promesa']<500000)]
consolidado_cred['fecha_promesa'] = pd.to_datetime(consolidado_cred['fecha_promesa'],dayfirst=False)
consolidado_cred.replace('CONTACTO','EXTRA',inplace=True)

In [53]:
pagos_cred3 = pd.merge(pagos_cred,consolidado_cred[['credito','nombre_usuario','fecha_promesa','monto_promesa','ARANCEL']],how='left',left_on=['ID'],right_on=['credito'])
pagos_cred3['ARANCEL'].fillna(np.mean(pagos_cred3['ARANCEL']),inplace=True)
pagos_cred3['PagoFacturacion'] = [x*y for x,y in zip(pagos_cred3['ARANCEL'],pagos_cred3['IMPORTE'])]
pagos_cred3['division'] = 'EXTRA'
pagos_cred3 = pagos_cred3.drop_duplicates(subset=['ID','IMPORTE'])

In [54]:
#Obtenemos datos de las promesas de Bradesco
prom_cred_div_mes = pd.pivot_table(consolidado_cred,index=['dictamen'],values=['monto_promesa'], aggfunc=['count',np.sum], dropna=False)
prom_cred_div_mes = pd.DataFrame(prom_cred_div_mes.to_records())
prom_cred_div_mes.columns = ['Division','NumeroPromesas','SumaPromesas']
prom_cred_div_mes

,Division,NumeroPromesas,SumaPromesas
0,EXTRA,93,276331.16


In [55]:
#Obtenemos las promesas restantes a partir del día de hoy
restantes_cred = consolidado_cred.loc[consolidado_cred['fecha_promesa']>=ayer]

promesas_restantes_cred = pd.pivot_table(restantes_cred,index=['dictamen'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_cred = pd.DataFrame(promesas_restantes_cred.to_records())
promesas_restantes_cred.columns = ['Division','PromesasRestantes']
promesas_restantes_cred.fillna(0,inplace=True)
promesas_restantes_cred

,Division,PromesasRestantes
0,EXTRA,101714.81


In [56]:
pagos_totales_cred = pd.pivot_table(pagos_cred3,index=['division'],values=['IMPORTE','PagoFacturacion'],aggfunc=['count',np.sum],dropna=False)
pagos_totales_cred = pd.DataFrame(pagos_totales_cred.to_records())
pagos_totales_cred = pagos_totales_cred.iloc[:,[0,1,3,4]]
pagos_totales_cred.columns = ['Division','NumeroPagos','SumaPagos','Facturacion']
pagos_totales_cred.fillna(0,inplace=True)
#pagos_totales_cred[['SumaPagos','SumaFacturacion']] = pagos_totales_cred[['SumaPagos','SumaFacturacion']].applymap("${0:,.2f}".format)
pagos_totales_cred['FechaFact'] = max(pagos_cred3['FECHA'])
pagos_totales_cred.insert(loc=0,column='Cliente',value='Credifiel')
pagos_totales_cred

,Cliente,Division,NumeroPagos,SumaPagos,Facturacion,FechaFact
0,Credifiel,EXTRA,34,183076.96,40253.956961,2021-01-14


In [57]:
# pagos_cred3['Cliente'] = 'Credifiel'
# pagos_totales_cred2 = pd.pivot_table(pagos_cred3,index=['Cliente'],values=['IMPORTE','PagoFacturacion'],aggfunc=['count',np.sum],dropna=False)
# pagos_totales_cred2 = pd.DataFrame(pagos_totales_cred2.to_records())
# pagos_totales_cred2 = pagos_totales_cred2.iloc[:,[0,1,3,4]]
# pagos_totales_cred2.columns = ['Division','NumeroPagos','SumaPagos','Facturacion']
# pagos_totales_cred2.fillna(0,inplace=True)
# # pagos_totales_cred[['SumaPagos','SumaFacturacion']] = pagos_totales_cred[['SumaPagos','SumaFacturacion']].applymap("${0:,.2f}".format)
# pagos_totales_cred2['FechaFact'] = max(pagos_cred3['FECHA'])
# pagos_totales_cred2

In [58]:
final_cred = pd.merge(pagos_totales_cred,prom_cred_div_mes,how='left',on=['Division'])
final_cred = pd.merge(final_cred,promesas_restantes_cred,how='left',on=['Division'])
final_cred.fillna(0,inplace=True)
final_cred = final_cred.replace('EXTRA','Extrajudicial')
final_cred = final_cred.iloc[:,[0,1,4,5,8]]
final_cred

,Cliente,Division,Facturacion,FechaFact,PromesasRestantes
0,Credifiel,Extrajudicial,40253.956961,2021-01-14,101714.81


# Banco Azteca

In [59]:
#Hacemos la consulta referente a las gestiones de Banco Azteca
gestion_baz = cs.query_naucalpan('baz_rpt_gestiones_detallado',mes,today)
gestion_baz.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_baz = gestion_baz.loc[gestion_baz['dictamen']=='PROMESA']
gestion_baz['monto_promesa'] = pd.to_numeric(gestion_baz['monto_promesa'])
gestion_baz = gestion_baz.loc[gestion_baz['fecha_promesa'].map(str).str[:4].map(int)<2100]
gestion_baz['fecha_promesa'] = pd.to_datetime(gestion_baz['fecha_promesa'],dayfirst=False)
#gestion_baz.head(3)

In [60]:
#Obtenemos datos de las promesas de Banco Azteca
gestion_baz = gestion_baz.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
prom_baz_div_mes = pd.pivot_table(gestion_baz,index=['dictamen'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
prom_baz_div_mes = pd.DataFrame(prom_baz_div_mes.to_records())
prom_baz_div_mes = prom_baz_div_mes.iloc[:,[0,1,2]]
prom_baz_div_mes.columns = ['Division','NumeroPromesas','SumaPromesas']
prom_baz_div_mes.fillna(0,inplace=True)
prom_baz_div_mes = prom_baz_div_mes.replace('PROMESA','COBRANZA')
prom_baz_div_mes

,Division,NumeroPromesas,SumaPromesas
0,COBRANZA,1516,1355664.0


In [61]:
#Obtenemos las promesas restantes a partir del día de ayer
restantes_baz = gestion_baz.loc[(gestion_baz['fecha_promesa']>=ayer)&(gestion_baz['fecha_promesa']<=fin_mes)]

promesas_restantes_baz = pd.pivot_table(restantes_baz,index=['dictamen'],values=['monto_promesa'],aggfunc=[np.sum],dropna=False)
promesas_restantes_baz = pd.DataFrame(promesas_restantes_baz.to_records())
promesas_restantes_baz.columns = ['Division','PromesasRestantes']
promesas_restantes_baz = promesas_restantes_baz.replace('PROMESA','COBRANZA')
promesas_restantes_baz.fillna(0,inplace=True)
promesas_restantes_baz

,Division,PromesasRestantes
0,COBRANZA,375880.0


In [62]:
pagos_baz = pd.concat([pagos_baz_26,pagos_baz_recovery],ignore_index=True)
pagos_baz = pagos_baz.loc[pagos_baz['CARGO AUTOMATICO']=='NO']
pagos_baz['Recuperado'] = pagos_baz['RECUPERACION DE CAPITAL']+pagos_baz['RECUPERACION DE MORATORIOS']
pagos_baz['Cliente'] = 'Cobranza'

In [63]:
pagos_totales_baz = pd.pivot_table(fact_baz,index=['Cliente'],values=['Total Recuperado','Total Comisión'],aggfunc=[np.sum],dropna=False)
pagos_totales_baz = pd.DataFrame(pagos_totales_baz.to_records())
pagos_totales_baz.columns = ['Cliente','Recuperado','SumaPagos']
porcentaje_prom_arancel_baz = pagos_totales_baz['Recuperado']/pagos_totales_baz['SumaPagos']
porcentaje_prom_arancel = porcentaje_prom_arancel_baz.item()
porcentaje_prom_arancel

0.16480033933839525

In [64]:
pagos_baz['Facturacion'] = [x*porcentaje_prom_arancel for x in pagos_baz['Recuperado']]
# pagos_baz['Facturacion'] = pagos_baz['Arancel']*pagos_baz['Recuperado']
pagos_fact_baz = pd.pivot_table(pagos_baz,index=['Cliente'],values=['Facturacion'],aggfunc=[np.sum])
pagos_fact_baz = pd.DataFrame(pagos_fact_baz.to_records())
pagos_fact_baz.columns = ['Division','Facturacion']
pagos_fact_baz['FechaFact'] = max(pagos_baz['DIA'])
pagos_fact_baz.insert(loc=0,column='Cliente',value='Banco Azteca')
pagos_fact_baz

,Cliente,Division,Facturacion,FechaFact
0,Banco Azteca,Cobranza,251711.423896,2021-01-20


In [65]:
toadd = [pagos_fact_baz,promesas_restantes_baz]
final_baz = pd.concat(toadd,axis=1)
final_baz = final_baz.iloc[:,[0,1,2,3,5]]
final_baz

,Cliente,Division,Facturacion,FechaFact,PromesasRestantes
0,Banco Azteca,Cobranza,251711.423896,2021-01-20,375880.0


# TORREON

# Oriflame

In [66]:
reporte_pagos_oriflame = pd.pivot_table(pagos_oriflame,index=['Cliente'],values=['Factura'],aggfunc=np.sum)
reporte_pagos_oriflame = pd.DataFrame(reporte_pagos_oriflame.to_records())
reporte_pagos_oriflame.columns = ['Cliente','Facturacion']
reporte_pagos_oriflame['FechaFact'] = max(pagos_oriflame['FECHA_PAGO'])
reporte_pagos_oriflame['FechaFact'] = pd.to_datetime(reporte_pagos_oriflame['FechaFact'],dayfirst=True)
reporte_pagos_oriflame

,Cliente,Facturacion,FechaFact
0,Oriflame,17714.686,2021-01-11


# Arabela

In [67]:
reporte_pagos_arabela = pd.pivot_table(pagos_arabela,index=['Cliente'],values=['factura'],aggfunc=np.sum)
reporte_pagos_arabela = pd.DataFrame(reporte_pagos_arabela.to_records())
reporte_pagos_arabela.columns = ['Cliente','Facturacion']
reporte_pagos_arabela['FechaFact'] = max(pagos_arabela['FechaEntrega'])
reporte_pagos_arabela['FechaFact'] = pd.to_datetime(reporte_pagos_arabela['FechaFact'],dayfirst=True)
reporte_pagos_arabela

,Cliente,Facturacion,FechaFact
0,Arabela,30832.8702,2021-01-12


# Invex

In [68]:
reporte_pagos_invex = pd.pivot_table(pagos_invex,index=['Cliente'],values=['AMT_TRAN','HONORARIO'],aggfunc=[np.sum,np.mean])
reporte_pagos_invex = pd.DataFrame(reporte_pagos_invex.to_records())
reporte_pagos_invex = reporte_pagos_invex.iloc[:,[0,1,4]]
reporte_pagos_invex.columns = ['Cliente','Pago','Honorario']
reporte_pagos_invex['Facturacion'] = reporte_pagos_invex['Pago']*reporte_pagos_invex['Honorario']
reporte_pagos_invex = reporte_pagos_invex.iloc[:,[0,3]]
reporte_pagos_invex['FechaFact'] = max(pagos_invex['TRAN_POST_DT'])
reporte_pagos_invex['FechaFact'] = pd.to_datetime(reporte_pagos_invex['FechaFact'],dayfirst=True)
reporte_pagos_invex

,Cliente,Facturacion,FechaFact
0,Invex_REL,158078.144,1970-01-01 00:00:00.000044211
1,Invex_RLA,212830.868,1970-01-01 00:00:00.000044211
2,Invex_RLN,91735.102,1970-01-01 00:00:00.000044211


# Torreon

In [69]:
final_torreon = pd.DataFrame([['Avon',42681,990000,8],['Oriflame',22307,65000,17],['Invex',462644,1095000,15],['Arabela',44315,95000,17],['Cimaco',547,3500,10],['Cristian Lay',0,6000,1]])
final_torreon.columns = ['Cliente','Facturacion','Meta','Dia']
final_torreon.insert(loc=0,column='Sucursal',value='Torreon')
final_torreon['FechaFact'] = [pd.to_datetime(str(x)+'/01/2021',dayfirst=True) for x in final_torreon['Dia']]
final_torreon.drop('Dia',axis=1,inplace=True)
final_torreon

,Sucursal,Cliente,Facturacion,Meta,FechaFact
0,Torreon,Avon,42681,990000,2021-01-08
1,Torreon,Oriflame,22307,65000,2021-01-17
2,Torreon,Invex,462644,1095000,2021-01-15
3,Torreon,Arabela,44315,95000,2021-01-17
4,Torreon,Cimaco,547,3500,2021-01-10
5,Torreon,Cristian Lay,0,6000,2021-01-01


# Guatemala

In [70]:
final_guatemala = pd.DataFrame([['Avon G',69940,250000,14],['Arabela G',808,5100,11],['Banco Azteca',6047,52000,17],['Banco Credomatic',2222,17000,12],['Belcorp',2010,5000,13],['Banco Azteca',37152,115000,18]])
final_guatemala.columns = ['Cliente','Facturacion','Meta','Dia']
final_guatemala.insert(loc=0,column='Sucursal',value='Guatemala')
final_guatemala['Facturacion'] = final_guatemala['Facturacion']*quetzal
final_guatemala['Meta'] = final_guatemala['Meta']*quetzal
final_guatemala['FechaFact'] = [pd.to_datetime(str(x)+'/01/2021',dayfirst=True) for x in final_guatemala['Dia']]
final_guatemala.drop('Dia',axis=1,inplace=True)
final_guatemala

,Sucursal,Cliente,Facturacion,Meta,FechaFact
0,Guatemala,Avon G,177647.60,635000.0,2021-01-14
1,Guatemala,Arabela G,2052.32,12954.0,2021-01-11
2,Guatemala,Banco Azteca,15359.38,132080.0,2021-01-17
3,Guatemala,Banco Credomatic,5643.88,43180.0,2021-01-12
4,Guatemala,Belcorp,5105.40,12700.0,2021-01-13
5,Guatemala,Banco Azteca,94366.08,292100.0,2021-01-18


# Panama

In [71]:
final_panama = pd.DataFrame([['Cable Onda',1436,2800,14],['Jamar',2500,8000,17],['Banco Azteca',4419,4000,16],['Adel Panama',893,3000,10],['Cable Onda',627,2000,14],['Banco Azteca',31000,34000,16],['Avon',485,2000,15],['Avon',1250,1250,30],['Rodelag',0,3500,1]])
final_panama.columns = ['Cliente','Facturacion','Meta','Dia']
final_panama.insert(loc=0,column='Sucursal',value='Panama')
final_panama['Facturacion'] = final_panama['Facturacion']*dolar
final_panama['Meta'] = final_panama['Meta']*dolar
final_panama['FechaFact'] = [pd.to_datetime(str(x)+'/01/2021',dayfirst=True) for x in final_panama['Dia']]
final_panama.drop('Dia',axis=1,inplace=True)
final_panama

,Sucursal,Cliente,Facturacion,Meta,FechaFact
0,Panama,Cable Onda,28432.8,55440.0,2021-01-14
1,Panama,Jamar,49500.0,158400.0,2021-01-17
2,Panama,Banco Azteca,87496.2,79200.0,2021-01-16
3,Panama,Adel Panama,17681.4,59400.0,2021-01-10
4,Panama,Cable Onda,12414.6,39600.0,2021-01-14
5,Panama,Banco Azteca,613800.0,673200.0,2021-01-16
6,Panama,Avon,9603.0,39600.0,2021-01-15
7,Panama,Avon,24750.0,24750.0,2021-01-30
8,Panama,Rodelag,0.0,69300.0,2021-01-01


# Pagos sin promesa asignada

In [72]:
#Pagos de bradesco que no tienen promesa asignada
gestion_brad_mes2 = gestion_brad_mes.drop_duplicates(subset='NUMERO_TARJETA')
pagos_noasig_brad = pd.merge(pagos_brad,gestion_brad_mes2[['NUMERO_TARJETA','nombre_credito']],how='left',on='NUMERO_TARJETA')
pagos_noasig_brad = pagos_noasig_brad.loc[pagos_noasig_brad['nombre_credito'].isnull()]
pagos_noasig_brad_pivot = pd.pivot_table(pagos_noasig_brad,index=['TIPO_ASIGANCION'],values=['MONTO'],aggfunc=[np.sum,'count'])
pagos_noasig_brad_pivot = pd.DataFrame(pagos_noasig_brad_pivot.to_records())
pagos_noasig_brad_pivot.columns = ['Division','RecuperadoSinPromesa','PagosNoPromesa']
pagos_noasig_brad_pivot

,Division,RecuperadoSinPromesa,PagosNoPromesa
0,A,40420.00,6
1,B,21817.00,2
2,C0 4PV,129917.32,106
3,C0 5PV,62674.04,58
4,C0 6PV,43993.50,35
5,C1,91347.00,63
6,C2,43631.84,34
7,C3,5899.00,8
8,C4,640.00,1


In [74]:
#Pagos de Liverpool que no tienen promesa asignada
gestion_liv_mes2 = gestion_liv_mes.drop_duplicates(subset='credito')
pagos_noasig_liv = pd.merge(pagos_liv2,gestion_liv_mes2[['credito','nombre_credito']],left_on='Cuenta',right_on='credito',how='left')
pagos_noasig_liv = pagos_noasig_liv.loc[pagos_noasig_liv['nombre_credito'].isnull()]
pagos_noasig_liv_pivot = pd.pivot_table(pagos_noasig_liv,index=['segmento'],values=['Monto'],aggfunc=[np.sum,'count'])
pagos_noasig_liv_pivot = pd.DataFrame(pagos_noasig_liv_pivot.to_records())
pagos_noasig_liv_pivot.columns = ['Division','RecuperadoSinPromesa','PagosNoPromesa']
pagos_noasig_liv_pivot

,Division,RecuperadoSinPromesa,PagosNoPromesa
0,Cof,28053.00,8
1,Dictamen,4305.00,4
2,Nacional2,63175.99,35
3,Penacional,59606.00,20
4,Suburbia,20292.83,24


In [75]:
#Pagos de Liverpool que no tienen promesa asignada
# gestion_cred2 = gestion_cred.drop_duplicates(subset='credito')
# pagos_noasig_cred = pd.merge(pagos_cred,gestion_cred2[['credito','nombre_credito']],left_on='ID',right_on='credito',how='left')
# pagos_noasig_cred = pagos_noasig_cred.loc[pagos_noas102281.0
# 4	C0 6PV	ig_cred['nombre_credito'].isnull()]
# pagos_noasig_cred['Cliente'] = 'Extrajudicial'
# pagos_noasig_cred_pivot = pd.pivot_table(pagos_noasig_cred,index=['Cliente'],values=['IMPORTE'],aggfunc=[np.sum,'count'])
# pagos_noasig_cred_pivot = pd.DataFrame(pagos_noasig_cred_pivot.to_records())
# pagos_noasig_cred_pivot.columns = ['Division','RecuperadoSinPromesa','PagosNoPromesa']
# pagos_noasig_cred_pivot

,Division,RecuperadoSinPromesa,PagosNoPromesa
0,Extrajudicial,124868.26,16


In [76]:
list1 = [pagos_noasig_cred_pivot,pagos_noasig_liv_pivot,pagos_noasig_brad_pivot]
pagos_noasig_naucalpan = pd.concat(list1,ignore_index=True)
pagos_noasig_naucalpan = pagos_noasig_naucalpan.iloc[:,:-1]
pagos_noasig_naucalpan

,Division,RecuperadoSinPromesa
0,Extrajudicial,124868.26
1,Cof,28053.00
2,Dictamen,4305.00
3,Nacional2,63175.99
4,Penacional,59606.00
5,Suburbia,20292.83
6,A,40420.00
7,B,21817.00
8,C0 4PV,129917.32
9,C0 5PV,62674.04


# Union Naucalpan

In [98]:
toadd_nau = [final_liv,final_brad,final_cred,final_baz]
final_naucalpan = pd.concat(toadd_nau,ignore_index=True)
final_naucalpan = pd.merge(final_naucalpan,pagos_noasig_naucalpan,on='Division',how='left')
final_naucalpan.insert(loc=0,column='Sucursal',value='Naucalpan')
final_naucalpan.insert(loc=0,column='Pais',value='Mexico')
final_naucalpan['TendenciaCierre1'] = [(30/x.day)*y for x,y in zip(final_naucalpan['FechaFact'],final_naucalpan['Facturacion'])]
final_naucalpan.fillna(0,inplace=True)
final_naucalpan

,Pais,Sucursal,Cliente,Division,Facturacion,FechaFact,Recuperado,RestSigArancel,PromesasRestantes,RecuperadoSinPromesa,TendenciaCierre1
0,Mexico,Naucalpan,Liverpool,Cof,90077.680000,2021-01-17,180155.37,419844.63,25326.00,28053.00,158960.611765
1,Mexico,Naucalpan,Liverpool,Dictamen,2672.160000,2021-01-17,16701.00,2170799.00,0.00,4305.00,4715.576471
2,Mexico,Naucalpan,Liverpool,Nacional2,142921.200000,2021-01-17,357302.99,1267697.01,76747.00,63175.99,252213.882353
3,Mexico,Naucalpan,Liverpool,Penacional,111875.680000,2021-01-17,399556.00,671872.60,75873.00,59606.00,197427.670588
4,Mexico,Naucalpan,Liverpool,Suburbia,23085.000000,2021-01-17,153899.98,1846100.02,56337.10,20292.83,40738.235294
5,Mexico,Naucalpan,Bradesco,A,20272.950000,2021-01-19,75085.00,185105.00,10000.00,40420.00,32009.921053
6,Mexico,Naucalpan,Bradesco,B,6341.000000,2021-01-19,25364.00,189719.00,2000.00,21817.00,10012.105263
7,Mexico,Naucalpan,Bradesco,C0 4PV,29163.560000,2021-01-19,416622.32,788081.68,207865.00,129917.32,46047.726316
8,Mexico,Naucalpan,Bradesco,C0 5PV,15198.130000,2021-01-19,168868.07,493400.93,102281.00,62674.04,23997.047368
9,Mexico,Naucalpan,Bradesco,C0 6PV,14582.450000,2021-01-19,145824.50,661347.50,65738.00,43993.50,23024.921053


In [78]:
pdList = [pagos_totales_liv2,pagos_totales_brad2,pagos_totales_cred,pagos_fact_baz]
final_naucalpan2 = pd.concat(pdList,ignore_index=True)
final_naucalpan2 = final_naucalpan2.iloc[:,:-3]
final_naucalpan2 = pd.merge(final_naucalpan2,meta_naucalpan,on=['Cliente'],how='left')
final_naucalpan2.insert(loc=0,column='Sucursal',value='Naucalpan')
final_naucalpan2

,Sucursal,Cliente,Facturacion,FechaFact,Meta
0,Naucalpan,Liverpool,370631.720000,2021-01-17,1150000
1,Naucalpan,Bradesco,176266.488300,2021-01-19,750000
2,Naucalpan,Credifiel,40253.956961,2021-01-14,100000
3,Naucalpan,Banco Azteca,251711.423896,2021-01-20,1000000


In [79]:
toadd_final = [final_naucalpan2,final_torreon,final_guatemala,final_panama]
final_sucursales = pd.concat(toadd_final,ignore_index=True)
final_sucursales['Avance%'] = final_sucursales['Facturacion']/final_sucursales['Meta']
final_sucursales['MetaDia'] = ((final_sucursales['FechaFact'].dt.day/30)*final_sucursales['Meta']).round()
final_sucursales['TendenciaCierre'] = [(30/x.day)*y for x,y in zip(final_sucursales['FechaFact'],final_sucursales['Facturacion'])]
final_sucursales['Semaforo'] = np.where(final_sucursales['MetaDia']<=final_sucursales['Facturacion'],'verde',
                                    np.where((final_sucursales['MetaDia']*.75<=final_sucursales['Facturacion']) &                                                                                    (final_sucursales['MetaDia']>final_sucursales['Facturacion']),'amarillo','rojo'))
final_sucursales
fecha_fact = datetime.strftime(max(final_sucursales['FechaFact']),'%d-%m-%Y')
final_sucursales

,Sucursal,Cliente,Facturacion,FechaFact,Meta,Avance%,MetaDia,TendenciaCierre,Semaforo
0,Naucalpan,Liverpool,370631.720000,2021-01-17,1150000.0,0.322288,651667.0,6.540560e+05,rojo
1,Naucalpan,Bradesco,176266.488300,2021-01-19,750000.0,0.235022,475000.0,2.783155e+05,rojo
2,Naucalpan,Credifiel,40253.956961,2021-01-14,100000.0,0.402540,46667.0,8.625848e+04,amarillo
3,Naucalpan,Banco Azteca,251711.423896,2021-01-20,1000000.0,0.251711,666667.0,3.775671e+05,rojo
4,Torreon,Avon,42681.000000,2021-01-08,990000.0,0.043112,264000.0,1.600538e+05,rojo
5,Torreon,Oriflame,22307.000000,2021-01-17,65000.0,0.343185,36833.0,3.936529e+04,rojo
6,Torreon,Invex,462644.000000,2021-01-15,1095000.0,0.422506,547500.0,9.252880e+05,amarillo
7,Torreon,Arabela,44315.000000,2021-01-17,95000.0,0.466474,53833.0,7.820294e+04,amarillo
8,Torreon,Cimaco,547.000000,2021-01-10,3500.0,0.156286,1167.0,1.641000e+03,rojo
9,Torreon,Cristian Lay,0.000000,2021-01-01,6000.0,0.000000,200.0,0.000000e+00,rojo


In [80]:
# writer = pd.ExcelWriter(''+ruta_guardar+'/Avance Pagos '+fecha_fact+'.xlsx',engine='xlsxwriter')
# final_sucursales.to_excel(writer,'FactCliente',index=False,header=True)
# final_naucalpan.to_excel(writer,'Naucalpan',index=False,header=True)

# writer.save()